In [1]:
!pip install -q langchain sentence-transformers datasets langchain-google-genai ragas unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.3/122.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
from tqdm.notebook import tqdm
import os
import pandas as pd
from typing import Optional, List, Tuple
import json

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness

pd.set_option("display.max_colwidth", None)

In [3]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

## Generate Synthetic Test Set

### Documents Loading

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
document_folder = "gdrive/MyDrive/LLM01 - RắnGPT/Data/Data as Documents"

In [6]:
from langchain.document_loaders import DirectoryLoader
loader = DirectoryLoader(document_folder)
documents = loader.load()
for document in documents:
    document.metadata['filename'] = document.metadata['source']

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


### Data Generation using `ragas` (failed)

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.run_config import RunConfig

# Init generator
generator_llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)
critic_llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)
# generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
# critic_llm = ChatOpenAI(model="gpt-4")
embeddings = embeddings

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

In [ ]:
# generate testset
testset = generator.generate_with_langchain_docs(documents, test_size=100, distributions={simple: 0.8, reasoning: 0.1, multi_context: 0.1})

In [ ]:
testset.to_pandas()

### Data Generation using code

**Reference**: https://huggingface.co/learn/cookbook/rag_evaluation

#### Prepare source documents

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = []
for doc in documents:
    docs_processed += text_splitter.split_documents([doc])

In [8]:
docs_processed[102]

Document(page_content='Tên Việt Nam: Rắn lục hoa cân. Tên Latinh / Tên khoa học: Tropidolaemus wagleri. Sinh học, sinh thái: Loài bò sát này sống ở các khu vực có nhiều cây bụi trong rừng thường xanh và cả rừng ngập mặn và cho tới độ cao khoảng 1.200m so với mặt biển. Hoạt động chủ yếu kiếm ăn đêm. Thức ăn là những loài lưỡng cư, thú gặm nhấm sống trong khu vực phân bố. Chúng thường sống trên cây, khá chậm chạp, nằm bất động trong một thời gian dài chờ đợi con mồi đi ngang qua và tấn công một cách bất ngờ. Là loài có nọc độc tế bào, rất nguy hiểm có thể gây chết người.. Phân bố: Trong nước: Đồng Nai, Sông Bé (Thủ Dầu Môt), Cà Mau (Theo ghi nhận của Orlov et al., 2003). Có thể loài này không có ở Việt Nam, nhưng chúng được buôn lậu đến Việt Nam và bị bắt, thu hồi, sau đó được các nhân viên bảo tồn tái thả ra môi trường tự nhiên. Nước ngoài: Loài này ghi nhận phân bố ở Nam Thai Lan, Tây Malaysia, một phần ở Indonesia và Singapo, Philippines..', metadata={'source': 'gdrive/MyDrive/LLM01 -

In [9]:
import google.generativeai as genai
genai.configure(api_key=GOOGLE_API_KEY)

llm_client = genai.GenerativeModel('gemini-pro')

def call_llm(model, question):

    return model.generate_content(question).text

In [ ]:
QA_generation_prompt = """
Your task is to write a factoid question and a detailed answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".
Your answer must be as detailed as possible given the context.
The question and answer MUST be in Vietnamese, the same language with the context.


Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

In [ ]:
import random
import time

N_GENERATIONS = 100  # We intentionally generate only 10 QA couples here for cost and time considerations

print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []
for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    # Generate QA couple
    try:
        output_QA_couple = call_llm(llm_client, QA_generation_prompt.format(context=sampled_context.page_content))
        time.sleep(0.2)
        question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0][:-1]
        answer = output_QA_couple.split("Answer: ")[-1]
        assert len(answer) < 300, "Answer is too long"
        outputs.append(
            {
                "contexts": sampled_context.page_content,
                "question": question,
                "ground_truths": answer
            }
        )
    except Exception as e:
        print(str(e))
        continue

Generating 100 QA couples...


  0%|          | 0/100 [00:00<?, ?it/s]

ERROR:tornado.access:503 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 4259.03ms


Answer is too long
The `response.text` quick accessor only works when the response contains a valid `Part`, but none was returned. Check the `candidate.safety_ratings` to see if the response was blocked.


#### Setup critique agents

In [ ]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independant this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

For instance, "What is the name of the checkpoint from which the ViT model is imported?" should receive a 1, since there is an implicit mention of a context, thus the question is not independant from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

In [ ]:
print("Generating critique for each QA couple...")
for output in tqdm(outputs):
    try:
      evaluations = {
          "groundedness": call_llm(
              llm_client,
              question_groundedness_critique_prompt.format(context=output["contexts"], question=output["question"]),
          ),
          # "relevance": call_llm(
          #     llm_client,
          #     question_relevance_critique_prompt.format(question=output["question"]),
          # ),
          "standalone": call_llm(
              llm_client,
              question_standalone_critique_prompt.format(question=output["question"]),
          ),
      }
    except:
      evaluations = {
          "groundedness": "Evaluation: nan\nTotal rating: -1",
          # "relevance": call_llm(
          #     llm_client,
          #     question_relevance_critique_prompt.format(question=output["question"]),
          # ),
          "standalone": "Evaluation: nan\nTotal rating: -1",
      }
    try:
        for criterion, evaluation in evaluations.items():
            score, eval = (
                int(evaluation.split("Total rating: ")[-1].strip()),
                evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],
            )
            output.update(
                {
                    f"{criterion}_score": score,
                    f"{criterion}_eval": eval,
                }
            )
    except Exception as e:
        continue

Generating critique for each QA couple...


  0%|          | 0/98 [00:00<?, ?it/s]

ERROR:tornado.access:500 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2968.64ms


In [ ]:
with open("/content/gdrive/MyDrive/LLM01 - RắnGPT/Code/Benchmark QA/synthetic_testset.json", "w") as output_file:
    json.dump(outputs, output_file, ensure_ascii=False, indent=2)

In [ ]:
import pandas as pd
import datasets

pd.set_option("display.max_colwidth", None)

generated_questions = pd.DataFrame.from_dict(outputs)

print("Evaluation dataset before filtering:")
display(
    generated_questions[
        [
            "question",
            "ground_truths",
            "contexts",
            "groundedness_score",
            # "relevance_score",
            "standalone_score",
        ]
    ]
)


eval_dataset = datasets.Dataset.from_pandas(generated_questions, split="train", preserve_index=False)
eval_dataset.to_csv('gdrive/MyDrive/LLM01 - RắnGPT/Code/Benchmark QA/benchmark_test_set_before_filtering.csv', index=False)

generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 4)
    # & (generated_questions["relevance_score"] >= 4)
    & (generated_questions["standalone_score"] >= 4)
]
print("============================================")
print("Final evaluation dataset:")
display(
    generated_questions[
        [
            "question",
            "ground_truths",
            "contexts",
            "groundedness_score",
            # "relevance_score",
            "standalone_score",
        ]
    ]
)

eval_dataset = datasets.Dataset.from_pandas(generated_questions, split="train", preserve_index=False)
eval_dataset.to_csv('gdrive/MyDrive/LLM01 - RắnGPT/Code/Benchmark QA/benchmark_test_set.csv', index=False)

Evaluation dataset before filtering:


,question,ground_truths,contexts,groundedness_score,standalone_score
0,Rắn sọc vàng có thể dài tới bao nhiêu mét?,"1,8 m","Tên Việt Nam: Rắn sọc vàng. Tên Latinh / Tên khoa học: Coelognathus flavolineatus. Sinh học, sinh thái: Loài rắn này thường sống ở gần những trang trại của nông dân. Chúng thường vào nhà dân để săn trứng gà và chuột. Đôi lúc thấy chúng trườn lên mái nhà gỗ. Thức ăn trong tự nhiên của chúng là thú gặm nhấm nhỏ, chim, ếch nhái, thằn lằn. Một lần con cái đẻ 5 - 12 trứng. Con đực dài đến 1.8 m. Phân bố: Trong nước: Lâm Đồng, Đồng Nai (Vườn quốc gia Cát Tiên, Thác Mai), Bình Phước (Vườn quốc gia Bù Gia Mập) Nước ngoài: Đảo Singapore, Thái Lan, Việt Nam, Campuchia, Ấn Độ (Quần đảo Andaman) Indonesia (Simalur, Billiton, Borneo, Sulawesi, Bangka, Belitung, Java, Kalimantan, Quần đảo Mentawai, Nias, Quần đảo Riau, Simeuleue, Sumatra, We, Bali); Vương quốc Bru-nây; Malaysia (Malaya và Đông Malaysia, Pulau Pangkor, Pulau Tioman), Myanmar..",2.0,5.0
1,Chỉ có một mục đích duy nhất của nọc độc rắn là tự vệ?,"Không, nọc độc của rắn có 2 mục đích, giết mồi để kiếm thức ăn và tự vệ.",Có phải tất cả loài rắn đều sử dụng nọc độc để tự vệ? - Không phải tất cả các loài rắn đều sử dụng nọc độc để tự vệ. Nọc độc được tạo ra với nhiệm vụ chính là giúp rắn đi săn. Tự vệ là cơ chế sự dụng thứ yếu của nọc độc.,1.0,5.0
2,Rắn hoa cỏ đỏ có răng nanh dạng nào?,Rắn hoa cỏ đỏ có răng nanh ẩn sâu phía sau các răng hàm (reared-fang),"Tên Việt Nam: Rắn hoa cỏ xiêm. Tên Latinh / Tên khoa học: Rhabdophis siamensis. Sinh học, sinh thái: Loài rắn hoạt động ban ngày này thích những nơi đất thấp có nhiều nước ở trong rừng và đồi núi có độ cao tới 1.780m. Các loài rắn độc thường có răng nanh tiêm nọc độc mọc ở phía ngoài (front-fang). Một số loài rắn độc khác có gai độc nằm sâu bên trong hàm nên rất khó phân biệt. Rắn hoa cỏ đỏ Rhabdophis subminiatus có răng nanh ẩn sâu phía sau các răng hàm (reared-fang), các răng nanh này lại rất nhỏ, ngắn và xếp gọn ở sâu trong hàm răng của rắn nên rất khó để phát hiện. Đó là lý do khiến những cú cắn của loài rắn này hiếm khi gây nguy hiểm cho con người. Chúng chỉ thực sự nguy hiểm khi cắn vào các bộ phận có gờ nhỏ như kẽ giữa ngón tay cái và ngón tay trỏ. Ngoài ra nó còn có một tuyến nọc khác gọi là tuyến Nuchal. Tuyến này có khả năng lọc và giữ lại các chất độc từ con mồi mà rắn ăn là loài Cóc nhà và Cóc rừng rồi tổng hợp, chuyển hóa chúng thành nọc độc của chính nó. Mỗi lần đẻ từ 5 - 17 trứng, thời gian ấp cho trứng nở từ 8 - 10 tuần; rắn con dài 13 - 19cm. Loài này đã phát hiện có tuyến độc khi bị rắn này cắn có những triệu chứng nghiêm trọng. Từ năm 1997 đến nay theo dữ liệu của bệnh viện Chợ Rẫy đã ghi nhận có 24 ca do Rắn hoa cỏ nhỏ Rhabdophis subminiatus tấn công. Loài Rắn này gây hậu quả nặng nề, nạn nhân bị cắn sẽ có các triệu chứng như: Rồi loạn đông máu như vết cắn rắn lục. Đã có 2 ca tử vong do loài rắn này vào năm 2009 và 2011.",1.0,5.0
3,Tên khoa học của rắn lục đuôi đỏ mắt nhỏ là gì?,Cryptelytrops macrops,"Tên Việt Nam: Rắn lục đuôi đỏ mắt nhỏ. Tên Latinh / Tên khoa học: Cryptelytrops macrops. Sinh học, sinh thái: Loài bò sát kiếm ăn đêm thường sống ở khu vực rừng thường xanh còn tốt. Xuất hiện quanh các khu vực vũng nước trong rừng để tìm kiếm thức ăn, săn mồi gồm các loài Lưỡng cư sống trong khu vực phân bố …Chỉ thường gặp vào mùa mưa, hầu như chưa gặp vào mùa khô. Sống cùng sinh cảnh với loài rắn lục đuôi đỏ Trimeresurus albolabris. Tên loài có nguồn gốc từ một từ Latin “rubens” có nghĩa là màu đỏ. Loài rắn độc nguy hiểm với nọc độc dạng tế bào, có thể gây chết người. Phân bố: Loài bò sát đặc hữu, mới phát hiện năm 2011 ở Vườn quốc gia Cát Tiện, Khu BTTN Vĩnh Cửu tỉnh Đồng Nai và thành phố Hồ Chí Minh. Các mẫu vật của loài này được thu ở sinh cảnh các vùng rừng trên núi đất thấp tới độ cao khoảng 500 m so với mực nước biển..",5.0,5.0
4,Rắn lột da bằng cách nào?,Rắn lột da bằng cách lộn ngược cơ thể từ bên trong ra,"Rắn lột da như thế nào, và chúng lột da bao lâu một lần? - Rắn lột da bằng cách

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Final evaluation dataset:


,question,ground_truths,contexts,groundedness_score,standalone_score
3,Tên khoa học của rắn lục đuôi đỏ mắt nhỏ là gì?,Cryptelytrops macrops,"Tên Việt Nam: Rắn lục đuôi đỏ mắt nhỏ. Tên Latinh / Tên khoa học: Cryptelytrops macrops. Sinh học, sinh thái: Loài bò sát kiếm ăn đêm thường sống ở khu vực rừng thường xanh còn tốt. Xuất hiện quanh các khu vực vũng nước trong rừng để tìm kiếm thức ăn, săn mồi gồm các loài Lưỡng cư sống trong khu vực phân bố …Chỉ thường gặp vào mùa mưa, hầu như chưa gặp vào mùa khô. Sống cùng sinh cảnh với loài rắn lục đuôi đỏ Trimeresurus albolabris. Tên loài có nguồn gốc từ một từ Latin “rubens” có nghĩa là màu đỏ. Loài rắn độc nguy hiểm với nọc độc dạng tế bào, có thể gây chết người. Phân bố: Loài bò sát đặc hữu, mới phát hiện năm 2011 ở Vườn quốc gia Cát Tiện, Khu BTTN Vĩnh Cửu tỉnh Đồng Nai và thành phố Hồ Chí Minh. Các mẫu vật của loài này được thu ở sinh cảnh các vùng rừng trên núi đất thấp tới độ cao khoảng 500 m so với mực nước biển..",5.0,5.0
4,Rắn lột da bằng cách nào?,Rắn lột da bằng cách lộn ngược cơ thể từ bên trong ra,"Rắn lột da như thế nào, và chúng lột da bao lâu một lần? - Rắn lột da bằng cách lộn ngược cơ thể từ bên trong ra. Chúng thường xuyên lột da trong suốt cuộc đời, đặc biệt là khi còn nhỏ. Tần suất lột da phụ thuộc vào loài, tuổi tác, môi trường sống và các yếu tố khác.",5.0,5.0
5,Rắn lục đầu trắng phân bố ở những tỉnh nào của Việt Nam?,"Cao Bằng, Bắc Kạn, Vĩnh Phúc, Lạng Sơn","Tên Việt Nam: Rắn lục đầu trắng. Tên Latinh / Tên khoa học: Azemiops feae. Sinh học, sinh thái: Rắn sống trong rừng, trên núi cao tới 1.000m còn gặp.. Phân bố: Trong nước: Cao Bằng (Cao Bằng), Bắc Kạn (Ngân Sơn), Vĩnh Phúc (Tam Đảo). Lạng Sơn. Thế giới: Bắc Mianma, Nam Trung Quốc.. Tình trạng: Số lượng ngoài tự nhiên còn rất ít. Hiện tại khu phân bố ước tính dưới 20.000km2, nơi cư trú: 500 - 2000km2, bị chia cắt nghiêm trọng, chỉ có 3 quần thể.. Biện pháp bảo vệ: Cấm săn bắt, buôn bán ở những khu vực loài này phân bố, cần thu một vài cặp về nghiên cứu nuôi sinh sản tại vườn thú..",5.0,5.0
7,Loài rắn biển này có bao nhiêu hàng vảy quanh cổ?,31–43 hàng,"Tên Việt Nam: Đẻn nhiều răng. Tên La tinh / Tên khoa học: Hydrophis caerulescens. Đặc điểm nhận dạng: Là loài rắn biển duy nhất có hơn 13 răng của xương hàm trên phía sau răng nanh độc (thường từ 14–18 răng). Hàng vảy quanh cổ 31–43; hàng vảy quanh thân 38–54; vảy bụng 253–334. Màu sắc: Cơ thể hơi xám xanh ở lưng, bụng hơi xám, có 40–60 khoanh màu đen hoặc đen sẫm, các khoanh này rộng hơn ở phần lưng và trở nên mờ nhạt theo tuổi, lúc này cơ thể hầu như hơi xanh xám.. Môi trường sống: Ở vùng biển cạn, trong các rạch bùn, rừng ngập mặn và đầm lầy (Murphy và cs, 1999).. Độc tố học: Độc tố thần kinh (Tu, 1987; 1988).. Sinh sản: Trứng thai - đẻ con mỗi lứa trung bình 6 con (Lemen & Voris,1981).. Thức ăn: Ít được biết.. Phân bố chung: Pakistan, Ấn Độ, biển Andaman, bán đảo In Đô - Mả Lai, Biển đông, Indonesia, biển Arafura và Carpentaria (David & Ineich, 1999). Phân bố ở Việt Nam: Vịnh Bắc Bộ (Kharin, 2004), cửa sông Cửu Long (Dương, 1978). Tình trạng: Ít gặp ở vùng biển Việt Nam..",5.0,5.0
8,Rắn sọc dưa có习 tính săn đuổi mồi chủ yếu là gì?,"Chuột, ngoài ra có cả thằn lằn và ếch nhái.","Tên Việt Nam: Rắn sọc dưa. Tên Latinh / Tên khoa học: Elaphe radiata. Sinh học, sinh thái: Loài rắn không độc sống trên cạn, song rất dữ, dễ bị kích thích, thường gặp ở đồng bằng và trung du, thường ẩn trong các hang chuột đã bỏ không, leo trèo giỏi trên các bờ rào, bụi cây um tùm, đôi khi trên mái nhà tranh. Khi gặp nguy hiểm có tập tính tự vệ đặc biệt. Dựng đứng một phần ba thân về phía trước lên khỏi mặt đất. Phần thân sau không cuộn tròn mà làm thành hình chữ S trên mặt đất. Cổ phình to theo chiều trước sau làm da cổ căng rộng để lộ rõ màu vàng và đen ở da giữa các vảy cổ. Miệng há rộng, hung hăng, doạ nạt, dữ tợn như tập tính của rắn ráo hoặc rắn hổ trâu khi chuẩn bị cắn vào kẻ thù. Bắt mồi cả vào ban ngày và ban đêm. Có tập tính săn đuổi mồi. Mồi chủ yếu là chuột, ngoài ra có

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

86824

In [ ]:
# Save
eval_dataset.to_csv('gdrive/MyDrive/LLM01 - RắnGPT/Code/Benchmark QA/benchmark_test_set.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

86824